# Books&Co - No, we will not be able to pay back the loan & afford the next book purchase order
## I used the ML Algorithm XGBoost 
    ### It is one of the most accurate predicting classification package although it has the potential to overfit.
## I looked at Accuracy to Validate the Model
    ### I ran out of time, but I used the training data to cross-validate the measure of accuracy to make sure the model is predicting properly.
## I used Python 3.5  to run my Python script
    ### Unfortunately, I was unable to save the script as a .py (I ran out of time)    
    DataScientist_Dia_Co-Osula_Lam.ipynb original_purchase_order.csv next_purchase_order.csv customer_features.csv product_features.csv last_month_assortment. csv next_month_assortment.csv

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
train = pd.read_csv(r"C:\Users\Osula\Documents\Recruitment\Interviews\Technical\Dia&Co - Data Scientist\last_month_assortment.csv")
test = pd.read_csv(r"C:\Users\Osula\Documents\Recruitment\Interviews\Technical\Dia&Co - Data Scientist\next_month_assortment.csv")
purchase = pd.read_csv(r"C:\Users\Osula\Documents\Recruitment\Interviews\Technical\Dia&Co - Data Scientist\original_purchase_order.csv")
customer = pd.read_csv(r"C:\Users\Osula\Documents\Recruitment\Interviews\Technical\Dia&Co - Data Scientist\customer_features.csv")
product = pd.read_csv(r"C:\Users\Osula\Documents\Recruitment\Interviews\Technical\Dia&Co - Data Scientist\product_features.csv")

In [3]:
train.head()

,customer_id,product_id,purchased
0,1000078,1730073,False
1,1000078,1470520,False
2,1000078,1861356,False
3,1000078,1240824,True
4,1000078,1155460,False


In [4]:
train = train.merge(purchase, on="product_id", how='left')
test = test.merge(purchase, on="product_id", how='left')
train.head()

,customer_id,product_id,purchased,quantity_purchased,cost_to_buy,retail_value
0,1000078,1730073,False,81,1.74,10.21
1,1000078,1470520,False,99,1.92,13.85
2,1000078,1861356,False,65,1.98,10.41
3,1000078,1240824,True,52,2.21,15.54
4,1000078,1155460,False,53,1.78,14.03


In [5]:
train = train.merge(product, on="product_id", how='left')
test = test.merge(product, on="product_id", how='left')
train.head()

,customer_id,product_id,purchased,quantity_purchased,cost_to_buy,retail_value,length,difficulty,fiction,genre
0,1000078,1730073,False,81,1.74,10.21,219,4,True,Drama
1,1000078,1470520,False,99,1.92,13.85,934,5,True,Romance
2,1000078,1861356,False,65,1.98,10.41,156,-1,True,Thriller
3,1000078,1240824,True,52,2.21,15.54,454,1,True,Classic
4,1000078,1155460,False,53,1.78,14.03,910,1,True,Thriller


In [6]:
train = train.merge(customer, on="customer_id", how='left')
test = test.merge(customer, on="customer_id", how='left')
print(train.head(), train.shape)
print("test =", test.shape)

   customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
0      1000078     1730073      False                  81         1.74   
1      1000078     1470520      False                  99         1.92   
2      1000078     1861356      False                  65         1.98   
3      1000078     1240824       True                  52         2.21   
4      1000078     1155460      False                  53         1.78   

   retail_value  length  difficulty  fiction     genre age_bucket  \
0         10.21     219           4     True     Drama      46-55   
1         13.85     934           5     True   Romance      46-55   
2         10.41     156          -1     True  Thriller      46-55   
3         15.54     454           1     True   Classic      46-55   
4         14.03     910           1     True  Thriller      46-55   

   is_returning_customer                                    favorite_genres  
0                   True  ['Classic', 'History', 'Self-Help', 

In [7]:
test.head()

,customer_id,product_id,quantity_purchased,cost_to_buy,retail_value,length,difficulty,fiction,genre,age_bucket,is_returning_customer,favorite_genres
0,1600785,1443029,71,2.27,12.88,266,1,True,Sci-Fi,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
1,1600785,1164138,97,2.23,14.11,749,2,True,Sci-Fi,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
2,1600785,1636511,51,1.61,10.77,278,1,True,Drama,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
3,1600785,1832547,95,2.09,13.60,268,2,True,Beach-Read,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
4,1600785,1237420,57,1.24,9.37,371,5,False,Self-Help,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."


In [8]:
train.isnull().sum()

customer_id                 0
product_id                  0
purchased                   0
quantity_purchased          0
cost_to_buy                 0
retail_value                0
length                      0
difficulty                  0
fiction                     0
genre                       0
age_bucket               1565
is_returning_customer       0
favorite_genres             0
dtype: int64

In [9]:
test.isnull().sum()

customer_id                 0
product_id                  0
quantity_purchased          0
cost_to_buy                 0
retail_value                0
length                      0
difficulty                  0
fiction                     0
genre                       0
age_bucket               2335
is_returning_customer       0
favorite_genres             0
dtype: int64

In [10]:
num = train.describe()
print(num)

        customer_id    product_id  quantity_purchased   cost_to_buy  \
count  3.600000e+04  3.600000e+04        36000.000000  36000.000000   
mean   1.299424e+06  1.510835e+06           74.462694      1.817113   
std    1.736275e+05  2.879044e+05           14.402848      0.348845   
min    1.000078e+06  1.003455e+06           50.000000      1.200000   
25%    1.150747e+06  1.265271e+06           62.000000      1.520000   
50%    1.298142e+06  1.520469e+06           74.000000      1.840000   
75%    1.451910e+06  1.762408e+06           87.000000      2.120000   
max    1.600731e+06  1.998756e+06           99.000000      2.400000   

       retail_value        length    difficulty  
count  36000.000000  3.600000e+04  36000.000000  
mean      12.046556  4.970584e+04      2.689889  
std        3.012084  2.161559e+05      1.719406  
min        6.220000  1.020000e+02     -1.000000  
25%        9.690000  3.390000e+02      1.000000  
50%       11.740000  5.780000e+02      3.000000  
75%       

In [11]:
nf = num.drop(['customer_id', 'product_id', 'quantity_purchased', 'cost_to_buy'], axis=1).columns
print(nf)
print(train[nf].skew())

Index(['retail_value', 'length', 'difficulty'], dtype='object')
retail_value    0.281779
length          4.170379
difficulty     -0.388037
dtype: float64


In [12]:
print(test[nf].skew())

retail_value    0.274114
length          4.070674
difficulty     -0.380050
dtype: float64


In [13]:
for n in train[nf].columns:
    robust = preprocessing.RobustScaler()
    minmax = preprocessing.MinMaxScaler()
    rob_mm = pipeline.Pipeline([('robust', robust), ('minmax', minmax)])
    rob_mm.fit(list(train[n].values) + list(test[n].values))
    train[n] = rob_mm.transform(list(train[n].values))
    test[n] = rob_mm.transform(list(test[n].values))
    
print('train: ', train.head(), train.shape)
print('test: ', test.head(), test.shape)

train:     customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
0      1000078     1730073      False                  81         1.74   
1      1000078     1470520      False                  99         1.92   
2      1000078     1861356      False                  65         1.98   
3      1000078     1240824       True                  52         2.21   
4      1000078     1155460      False                  53         1.78   

   retail_value    length  difficulty  fiction     genre age_bucket  \
0      0.298876  0.000117    0.833333     True     Drama      46-55   
1      0.571536  0.000832    1.000000     True   Romance      46-55   
2      0.313858  0.000054    0.000000     True  Thriller      46-55   
3      0.698127  0.000352    0.333333     True   Classic      46-55   
4      0.585019  0.000808    0.333333     True  Thriller      46-55   

   is_returning_customer                                    favorite_genres  
0                   True  ['Classic', 'His

C:\Users\Osula\Anaconda2\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:986: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Osula\Anaconda2\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:1008: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Osula\Anaconda2\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:986: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.r

In [14]:
# Fiction: True = 1 & False = 0
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(list(train['fiction'].values)+list(test['fiction'].values))
train['fiction'] = label_encoder.transform(list(train['fiction'].values))
test['fiction'] = label_encoder.transform(list(test['fiction'].values))
test.head()

,customer_id,product_id,quantity_purchased,cost_to_buy,retail_value,length,difficulty,fiction,genre,age_bucket,is_returning_customer,favorite_genres
0,1600785,1443029,71,2.27,0.498876,0.000164,0.333333,1,Sci-Fi,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
1,1600785,1164138,97,2.23,0.591011,0.000647,0.500000,1,Sci-Fi,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
2,1600785,1636511,51,1.61,0.340824,0.000176,0.333333,1,Drama,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
3,1600785,1832547,95,2.09,0.552809,0.000166,0.500000,1,Beach-Read,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
4,1600785,1237420,57,1.24,0.235955,0.000269,1.000000,0,Self-Help,56-65,False,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."


In [15]:
# Purchased: True = 1 & False = 0
train['purchased'] = label_encoder.fit_transform(list(train['purchased'].values))
train.head()

,customer_id,product_id,purchased,quantity_purchased,cost_to_buy,retail_value,length,difficulty,fiction,genre,age_bucket,is_returning_customer,favorite_genres
0,1000078,1730073,0,81,1.74,0.298876,0.000117,0.833333,1,Drama,46-55,True,"['Classic', 'History', 'Self-Help', 'Romance',..."
1,1000078,1470520,0,99,1.92,0.571536,0.000832,1.000000,1,Romance,46-55,True,"['Classic', 'History', 'Self-Help', 'Romance',..."
2,1000078,1861356,0,65,1.98,0.313858,0.000054,0.000000,1,Thriller,46-55,True,"['Classic', 'History', 'Self-Help', 'Romance',..."
3,1000078,1240824,1,52,2.21,0.698127,0.000352,0.333333,1,Classic,46-55,True,"['Classic', 'History', 'Self-Help', 'Romance',..."
4,1000078,1155460,0,53,1.78,0.585019,0.000808,0.333333,1,Thriller,46-55,True,"['Classic', 'History', 'Self-Help', 'Romance',..."


In [16]:
# Fiction: True = 1 & False = 0
label_encoder.fit(list(train['is_returning_customer'].values)+list(test['is_returning_customer'].values))
train['is_returning_customer'] = label_encoder.transform(list(train['is_returning_customer'].values))
test['is_returning_customer'] = label_encoder.transform(list(test['is_returning_customer'].values))
test.head()

,customer_id,product_id,quantity_purchased,cost_to_buy,retail_value,length,difficulty,fiction,genre,age_bucket,is_returning_customer,favorite_genres
0,1600785,1443029,71,2.27,0.498876,0.000164,0.333333,1,Sci-Fi,56-65,0,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
1,1600785,1164138,97,2.23,0.591011,0.000647,0.500000,1,Sci-Fi,56-65,0,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
2,1600785,1636511,51,1.61,0.340824,0.000176,0.333333,1,Drama,56-65,0,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
3,1600785,1832547,95,2.09,0.552809,0.000166,0.500000,1,Beach-Read,56-65,0,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."
4,1600785,1237420,57,1.24,0.235955,0.000269,1.000000,0,Self-Help,56-65,0,"['Romance', 'Biography', 'Classic', 'Pop-Psych..."


In [17]:
train['genre'].value_counts()

Beach-Read        4451
Romance           4283
Thriller          3913
Classic           3699
Drama             3616
Sci-Fi            3457
Biography         2738
Pop-Sci           2690
Pop-Psychology    2630
History           2514
Self-Help         2009
Name: genre, dtype: int64

In [18]:
train['age_bucket'].value_counts()

46-55    8025
56-65    6090
36-45    6070
26-35    5285
66+      5115
18-25    3555
0-17      295
Name: age_bucket, dtype: int64

In [19]:
train.groupby(['purchased', 'age_bucket'])['purchased'].value_counts()

purchased  age_bucket  purchased
0          0-17        0             217
           18-25       0            2497
           26-35       0            3370
           36-45       0            3772
           46-55       0            5077
           56-65       0            3896
           66+         0            3454
1          0-17        1              78
           18-25       1            1058
           26-35       1            1915
           36-45       1            2298
           46-55       1            2948
           56-65       1            2194
           66+         1            1661
Name: purchased, dtype: int64

In [20]:
label_encoder.fit(list(train['age_bucket'].values)+list(test['age_bucket'].values))
train['age_bucket'] = label_encoder.transform(list(train['age_bucket'].values))
test['age_bucket'] = label_encoder.transform(list(test['age_bucket'].values))
train['age_bucket'].value_counts()

4    8025
5    6090
3    6070
2    5285
6    5115
1    3555
7    1565
0     295
Name: age_bucket, dtype: int64

In [21]:
genre_dum = pd.get_dummies(train['genre'])
genre_dumt = pd.get_dummies(test['genre'])
print("train: ", genre_dum.head(), genre_dum.shape)
print("test: ", genre_dumt.head(), genre_dumt.shape)

train:     Beach-Read  Biography  Classic  Drama  History  Pop-Psychology  Pop-Sci  \
0           0          0        0      1        0               0        0   
1           0          0        0      0        0               0        0   
2           0          0        0      0        0               0        0   
3           0          0        1      0        0               0        0   
4           0          0        0      0        0               0        0   

   Romance  Sci-Fi  Self-Help  Thriller  
0        0       0          0         0  
1        1       0          0         0  
2        0       0          0         1  
3        0       0          0         0  
4        0       0          0         1   (36000, 11)
test:     Beach-Read  Biography  Classic  Drama  History  Pop-Psychology  Pop-Sci  \
0           0          0        0      0        0               0        0   
1           0          0        0      0        0               0        0   
2           0     

In [22]:
# append dummies
for i in genre_dum.columns:
    train['genre_'+ str(i)] = genre_dum[i]
    test['genre_'+ str(i)] = genre_dumt[i]

print("train: ", train.head(), train.shape)

train:     customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
0      1000078     1730073          0                  81         1.74   
1      1000078     1470520          0                  99         1.92   
2      1000078     1861356          0                  65         1.98   
3      1000078     1240824          1                  52         2.21   
4      1000078     1155460          0                  53         1.78   

   retail_value    length  difficulty  fiction     genre       ...        \
0      0.298876  0.000117    0.833333        1     Drama       ...         
1      0.571536  0.000832    1.000000        1   Romance       ...         
2      0.313858  0.000054    0.000000        1  Thriller       ...         
3      0.698127  0.000352    0.333333        1   Classic       ...         
4      0.585019  0.000808    0.333333        1  Thriller       ...         

   genre_Biography  genre_Classic genre_Drama  genre_History  \
0                0        

### Impute missing age_bucket variable

In [23]:
# new dataframe containing rows with known ages
new_train = train[~train.age_bucket.isin([7])]
print(new_train.head(), new_train.shape)
print(new_train.columns)
print(new_train['age_bucket'].value_counts())

   customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
0      1000078     1730073          0                  81         1.74   
1      1000078     1470520          0                  99         1.92   
2      1000078     1861356          0                  65         1.98   
3      1000078     1240824          1                  52         2.21   
4      1000078     1155460          0                  53         1.78   

   retail_value    length  difficulty  fiction     genre       ...        \
0      0.298876  0.000117    0.833333        1     Drama       ...         
1      0.571536  0.000832    1.000000        1   Romance       ...         
2      0.313858  0.000054    0.000000        1  Thriller       ...         
3      0.698127  0.000352    0.333333        1   Classic       ...         
4      0.585019  0.000808    0.333333        1  Thriller       ...         

   genre_Biography  genre_Classic genre_Drama  genre_History  \
0                0              0 

In [24]:
# new_train wil store all data with NaN in Age_Bucket
df = train[train.age_bucket.isin([7])]
print(df.head())
print(df['age_bucket'].value_counts())

    customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
70      1001483     1529979          1                  79         1.82   
71      1001483     1434159          0                  92         1.51   
72      1001483     1298229          1                  76         2.19   
73      1001483     1073542          0                  63         1.93   
74      1001483     1560221          0                  67         2.32   

    retail_value    length  difficulty  fiction           genre  \
70      0.379026  0.000591    0.666667        1          Sci-Fi   
71      0.445693  0.000055    0.500000        0  Pop-Psychology   
72      0.621723  0.000332    0.833333        1      Beach-Read   
73      0.405243  0.000802    0.666667        1      Beach-Read   
74      0.626966  0.000451    0.833333        1        Thriller   

         ...        genre_Biography  genre_Classic genre_Drama  genre_History  \
70       ...                      0              0           0   

In [25]:
# new dataframe containing rows with known ages
new_test = test[~test.age_bucket.isin([7])]
print(new_test.head(), new_test.shape)
print(new_test['age_bucket'].value_counts())
print(new_test.columns)

   customer_id  product_id  quantity_purchased  cost_to_buy  retail_value  \
0      1600785     1443029                  71         2.27      0.498876   
1      1600785     1164138                  97         2.23      0.591011   
2      1600785     1636511                  51         1.61      0.340824   
3      1600785     1832547                  95         2.09      0.552809   
4      1600785     1237420                  57         1.24      0.235955   

     length  difficulty  fiction       genre  age_bucket       ...        \
0  0.000164    0.333333        1      Sci-Fi           5       ...         
1  0.000647    0.500000        1      Sci-Fi           5       ...         
2  0.000176    0.333333        1       Drama           5       ...         
3  0.000166    0.500000        1  Beach-Read           5       ...         
4  0.000269    1.000000        0   Self-Help           5       ...         

   genre_Biography genre_Classic  genre_Drama  genre_History  \
0               

In [26]:
df_test = test[test.age_bucket.isin([7])]
print(df_test.head())
print(df_test['age_bucket'].value_counts())

     customer_id  product_id  quantity_purchased  cost_to_buy  retail_value  \
190      1602977     1087656                  69         1.25      0.208989   
191      1602977     1322176                  69         1.59      0.423970   
192      1602977     1707084                  54         1.26      0.119101   
193      1602977     1726401                  54         2.08      0.465918   
194      1602977     1151645                  99         1.24      0.286142   

       length  difficulty  fiction           genre  age_bucket  \
190  0.000200    0.833333        0         Pop-Sci           7   
191  0.000229    1.000000        1         Romance           7   
192  0.000598    0.500000        0       Biography           7   
193  0.000486    1.000000        1         Romance           7   
194  0.000069    0.333333        0  Pop-Psychology           7   

          ...        genre_Biography genre_Classic  genre_Drama  \
190       ...                      0             0           

In [27]:
X = new_train.iloc[:, [2,5,6,7,8,11,14,15,16,17,18,19,20,21,22,23]].values
y = new_train.iloc[:, 10].values
print(X,y)
print(new_train.iloc[:, [2,5,6,7,8,11,14,15,16,17,18,19,20,21,22,23]].columns)

[[  0.00000000e+00   2.98876404e-01   1.16954739e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   5.71535581e-01   8.31678141e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   3.13857678e-01   5.39791101e-05 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 ..., 
 [  1.00000000e+00   9.11610487e-01   6.42751255e-04 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   7.54307116e-01   6.74738876e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   1.82771536e-01   6.80736555e-04 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]] [4 4 4 ..., 3 3 3]
Index(['purchased', 'retail_value', 'length', 'difficulty', 'fiction',
       'is_returning_customer', 'genre_Biography', 'genre_Classic',
       'genre_Drama', 'genre_History', 'genre_Pop-Psychology', 'genre_Pop-Sci',
       'genre_Romance', 'genre_Sci-Fi', 'genre_Self-Help', 'genre_Thriller'],
    

### Compared accuracy of 4 classification algorithms to impute missing Age_Bucket variable

In [28]:
# Logistic Regression
clf = LogisticRegression()
clf.fit(X, y)
cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
scores = cross_val_score(clf, X, y, cv=4)
scores.mean()

0.23281549319508943

In [29]:
# Random Forest Classifier
clf = RandomForestClassifier(max_depth=8, random_state=0)
clf.fit(X, y)
cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
scores = cross_val_score(clf, X, y, cv=4)
scores.mean()

0.22628095789865738

In [30]:
# KNN
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X, y)
cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
scores = cross_val_score(clf, X, y, cv=4)
scores.mean()

0.17464724352318475

In [31]:
#SVM
clf = SVC(kernel='linear', C=1)
clf.fit(X, y)
cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
scores = cross_val_score(clf, X, y, cv=4)
print(scores.mean())
print(np.unique(y))

0.233047781232
[0 1 2 3 4 5 6]


In [32]:
# since SVM has the highest accuracy
clf = SVC(kernel='linear', C=1)
clf.fit(X, y)
X = df.iloc[:, [2,5,6,7,8,11,14,15,16,17,18,19,20,21,22,23]].values
pred = clf.predict(X)
print(np.unique(pred))
print(pred)

[4]
[4 4 4 ..., 4 4 4]


In [33]:
Xt = new_test.iloc[:, [4,5,6,7,10,12,13,14,15,16,17,18,19,20,21,22]].values
yt = new_test.iloc[:, 9].values
print(Xt,yt)
print(new_test.iloc[:, [4,5,6,7,10,12,13,14,15,16,17,18,19,20,21,22]].columns)

[[  4.98876404e-01   1.63936557e-04   3.33333333e-01 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  5.91011236e-01   6.46749708e-04   5.00000000e-01 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.40823970e-01   1.75931914e-04   3.33333333e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  2.98876404e-01   1.16954739e-04   8.33333333e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.47940075e-01   8.25680462e-04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.90636704e-01   2.08919148e-04   5.00000000e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]] [5 5 5 ..., 5 5 5]
Index(['retail_value', 'length', 'difficulty', 'fiction',
       'is_returning_customer', 'genre_Beach-Read', 'genre_Biography',
       'genre_Classic', 'genre_Drama', 'genre_History', 'genre_Pop-Psychology',
       'genre_Pop-Sci', 'genre_Romance', 'genre_Sci-Fi', 'genre_Self-Help',
       'genre_Th

In [34]:
clf = SVC(kernel='linear', C=1).fit(Xt, yt)
Xt = df_test.iloc[:, [4,5,6,7,10,12,13,14,15,16,17,18,19,20,21,22]].values
predt = clf.predict(Xt)

In [35]:
# Will use SVM to impute age_bucket since it has the highest accuracy score
y = df.iloc[:, 10].values
yt = df_test.iloc[:, 9].values
print(X,Xt)
print(df.iloc[:, [2,5,6,7,8,11,14,15,16,17,18,19,20,21,22,23]].columns)

[[  1.00000000e+00   3.79026217e-01   5.90771371e-04 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.45692884e-01   5.49787232e-05 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   6.21722846e-01   3.31871566e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  1.00000000e+00   4.10486891e-01   1.71933462e-04 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   3.76029963e-01   5.32793809e-04 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   5.67041199e-01   6.04765956e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]] [[  2.08988764e-01   1.99922630e-04   8.33333333e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.23970037e-01   2.28911411e-04   1.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.19101124e-01   5.97768664e-04   5.00000000e-01 ...,   0.00000000e+00
    0.00000000e+00   

In [36]:
df['age_bucket'] = pred
df_test['age_bucket'] = predt
print(df['age_bucket'].value_counts())
print(df_test['age_bucket'].value_counts())

4    1565
Name: age_bucket, dtype: int64
4    2335
Name: age_bucket, dtype: int64


C:\Users\Osula\Anaconda2\envs\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Osula\Anaconda2\envs\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
train = new_train.append(df)
test = new_test.append(df_test)
print(train.head(), train.shape)
print(train.isnull().sum())

   customer_id  product_id  purchased  quantity_purchased  cost_to_buy  \
0      1000078     1730073          0                  81         1.74   
1      1000078     1470520          0                  99         1.92   
2      1000078     1861356          0                  65         1.98   
3      1000078     1240824          1                  52         2.21   
4      1000078     1155460          0                  53         1.78   

   retail_value    length  difficulty  fiction     genre       ...        \
0      0.298876  0.000117    0.833333        1     Drama       ...         
1      0.571536  0.000832    1.000000        1   Romance       ...         
2      0.313858  0.000054    0.000000        1  Thriller       ...         
3      0.698127  0.000352    0.333333        1   Classic       ...         
4      0.585019  0.000808    0.333333        1  Thriller       ...         

   genre_Biography  genre_Classic genre_Drama  genre_History  \
0                0              0 

In [38]:
test.isnull().sum()

customer_id              0
product_id               0
quantity_purchased       0
cost_to_buy              0
retail_value             0
length                   0
difficulty               0
fiction                  0
genre                    0
age_bucket               0
is_returning_customer    0
favorite_genres          0
genre_Beach-Read         0
genre_Biography          0
genre_Classic            0
genre_Drama              0
genre_History            0
genre_Pop-Psychology     0
genre_Pop-Sci            0
genre_Romance            0
genre_Sci-Fi             0
genre_Self-Help          0
genre_Thriller           0
dtype: int64

In [39]:
age_dum = pd.get_dummies(train['age_bucket'])
age_dumt = pd.get_dummies(test['age_bucket'])
print("train: ", age_dum.head(), age_dum.shape)
print("test: ", age_dumt.head(), age_dumt.shape)

train:     0  1  2  3  4  5  6
0  0  0  0  0  1  0  0
1  0  0  0  0  1  0  0
2  0  0  0  0  1  0  0
3  0  0  0  0  1  0  0
4  0  0  0  0  1  0  0 (36000, 7)
test:     0  1  2  3  4  5  6
0  0  0  0  0  0  1  0
1  0  0  0  0  0  1  0
2  0  0  0  0  0  1  0
3  0  0  0  0  0  1  0
4  0  0  0  0  0  1  0 (56400, 7)


In [40]:
for i in age_dum.columns:
    train['age_'+ str(i)] = age_dum[i]
    test['age_'+ str(i)] = age_dumt[i]
    
print("test: ", test.head(), test.shape)

test:     customer_id  product_id  quantity_purchased  cost_to_buy  retail_value  \
0      1600785     1443029                  71         2.27      0.498876   
1      1600785     1164138                  97         2.23      0.591011   
2      1600785     1636511                  51         1.61      0.340824   
3      1600785     1832547                  95         2.09      0.552809   
4      1600785     1237420                  57         1.24      0.235955   

     length  difficulty  fiction       genre  age_bucket  ...    genre_Sci-Fi  \
0  0.000164    0.333333        1      Sci-Fi           5  ...               1   
1  0.000647    0.500000        1      Sci-Fi           5  ...               1   
2  0.000176    0.333333        1       Drama           5  ...               0   
3  0.000166    0.500000        1  Beach-Read           5  ...               0   
4  0.000269    1.000000        0   Self-Help           5  ...               0   

  genre_Self-Help  genre_Thriller  age_0  a

In [41]:
train = train.drop([
            'customer_id', 'product_id', 'quantity_purchased', 'cost_to_buy', 
            'genre', 'age_bucket', 'favorite_genres'], axis = 1)
print(train.head(), train.shape)
print(train.columns)

   purchased  retail_value    length  difficulty  fiction  \
0          0      0.298876  0.000117    0.833333        1   
1          0      0.571536  0.000832    1.000000        1   
2          0      0.313858  0.000054    0.000000        1   
3          1      0.698127  0.000352    0.333333        1   
4          0      0.585019  0.000808    0.333333        1   

   is_returning_customer  genre_Beach-Read  genre_Biography  genre_Classic  \
0                      1                 0                0              0   
1                      1                 0                0              0   
2                      1                 0                0              0   
3                      1                 0                0              1   
4                      1                 0                0              0   

   genre_Drama  ...    genre_Sci-Fi  genre_Self-Help  genre_Thriller  age_0  \
0            1  ...               0                0               0      0   
1     

In [42]:
y_train = train['purchased']
next = test.iloc[:, 0:2]
print(next)
test = test.drop([
            'customer_id', 'product_id', 'quantity_purchased', 'cost_to_buy', 
            'genre', 'age_bucket', 'favorite_genres'], axis = 1)
print(test.head(), test.shape)
print(test.columns)

       customer_id  product_id
0          1600785     1443029
1          1600785     1164138
2          1600785     1636511
3          1600785     1832547
4          1600785     1237420
5          1600852     1597824
6          1600852     1755017
7          1600852     1267131
8          1600852     1516839
9          1600852     1833307
10         1600959     1486209
11         1600959     1028567
12         1600959     1889979
13         1600959     1634459
14         1600959     1596114
15         1600969     1820289
16         1600969     1465869
17         1600969     1140363
18         1600969     1745803
19         1600969     1859893
20         1601034     1094131
21         1601034     1602884
22         1601034     1112685
23         1601034     1605396
24         1601034     1387146
25         1601055     1626869
26         1601055     1025311
27         1601055     1324552
28         1601055     1986746
29         1601055     1624722
...            ...         ...
55835   

## Final Models
### Wanted to compare ould have used XGBoost since predictions were more accuarte

In [43]:
# XGBoost
xgb_params = {
        'n_trees': 395,
        'eta': 0.0065,
        'max_depth': 3,
        'subsample': 0.50,
        'min_child_weight': 34,
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'base_score': 0.5,
        'silent': 1
}

In [44]:
dtrain = xgb.DMatrix(train.drop('purchased', axis = 1), y_train)
dtest = xgb.DMatrix(test)
print(dtrain.num_col(), dtrain.num_row())
print(dtest.num_col(), dtest.num_row())

23 36000
23 56400


In [45]:
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=1500,
                   #nfold=10,
                   early_stopping_rounds=25,
                   verbose_eval=50, 
                   show_stdv=False,
                   seed=2017
                  )

[0]	train-error:0.261472	test-error:0.264194
[50]	train-error:0.256667	test-error:0.258416
[100]	train-error:0.252986	test-error:0.254333
[150]	train-error:0.252361	test-error:0.253167
[200]	train-error:0.250361	test-error:0.252083
[250]	train-error:0.249181	test-error:0.250917
[300]	train-error:0.248083	test-error:0.249778
[350]	train-error:0.246556	test-error:0.2485
[400]	train-error:0.245375	test-error:0.248028


In [46]:
num_boost_rounds = len(cv_result)
num_boost_rounds

388

In [47]:
## Train model
model = xgb.train(
            dict(xgb_params, silent=0), 
            dtrain, num_boost_round=num_boost_rounds)

y_pred = model.predict(dtest)

In [48]:
print("Accuracy on Training Set: {}".format(accuracy_score(dtrain.get_label(), model.predict(dtrain).round(), normalize=False)))
print("Confusion Matrix on Training Set: {}".format(confusion_matrix(dtrain.get_label(), model.predict(dtrain).round())))

Accuracy on Training Set: 27136
Confusion Matrix on Training Set: [[19562  3759]
 [ 5105  7574]]


In [49]:
# Predict on the test set and save results.
output = pd.DataFrame({
                'customer_id': next.loc[:, 'customer_id'].astype(np.int32), 
                'product_id': next.loc[:, 'product_id'].astype(np.int32),     
                'purchased': y_pred})

output.to_csv('next_month_assortment.csv', index=False)